### UK Access, Watch, Reserve, and Other classification for antibiotics in dmd

The ["UK Access, Watch, Reserve, and Other classification for antibiotics"](https://www.gov.uk/government/publications/uk-aware-antibiotic-classification/uk-access-watch-reserve-and-other-classification-for-antibiotics-uk-aware-antibiotic-classification) categorises antibiotics into groups such as Access, Watch, and Reserve to guide healthcare professionals in optimising their use and mitigating antimicrobial resistance.

To facilitate research, it is helpful to link this categorisation to the [dm+d standard](https://www.nhsbsa.nhs.uk/pharmacies-gp-practices-and-appliance-contractors/dictionary-medicines-and-devices-dmd).

### Imports
We need to import some libaries to help with the code

In [1]:
import pandas as pd
import requests
from ebmdatalab import bq
import os

### Getting the AWaRe list
The AWaRe list is currently available on the following webpage. We can read the html page directly into Pandas to retrieve the first table.

In [2]:
# Set URL to scrape
url = "https://www.gov.uk/government/publications/uk-aware-antibiotic-classification/uk-access-watch-reserve-and-other-classification-for-antibiotics-uk-aware-antibiotic-classification"

# Read the first HTML table from the page into a DataFrame using the HTML content
df = pd.read_html(url)[0]

# Display the DataFrame
df

,Antibiotic,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category
0,Amikacin,Watch,Watch
1,Amoxicillin,Access,Access
2,Amoxicillin/ clavulanic-acid,Watch,Watch
3,Ampicillin,Access,Access
4,Azithromycin,Watch,Watch
...,...,...,...
85,Tigecycline,Reserve,Reserve
86,Tinidazole #,Other,Access
87,Tobramycin,Watch,Watch
88,Trimethoprim,Access,Access


### Cleaning up the list
The list contains some characters or identifiers that could break matching, we need to clean these up.

The # symbol is used to denote where category has changed, remove this

In [3]:
df['Antibiotic'] = df['Antibiotic'].str.replace(" #", "", regex=False)

We can review remaining rows that contain non-alphabetical characters

In [4]:
df[df['Antibiotic'].str.contains(r'[^A-Za-z ]', na=False)]

,Antibiotic,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category
2,Amoxicillin/ clavulanic-acid,Watch,Watch
6,Benzathine-benzylpenicillin,Access,Access
20,Ceftaroline-fosamil,Reserve,Reserve
22,Ceftazidime/ avibactam,Reserve,Reserve
23,Ceftobiprole-medocaril,Reserve,Reserve
24,Ceftolozane/ tazobactam,Reserve,Reserve
31,"Colistin, intravenous",Reserve,Reserve
32,"Colistin, oral",Reserve,Reserve
34,Dalfopristin/ quinupristin,Reserve,Reserve
46,"Fosfomycin, intravenous",Reserve,Reserve


Some rows specify a route. Sometimes there is a comma between antibiotic and route, sometimes there isn't. We can pull any specified route into a seperate column.

In [5]:
# List of routes to check.
route_list = ["oral", "intravenous"]

# Function to extract the route if the Antibiotic entry ends with a route.
def extract_route(antibiotic):
    for route in route_list:
        if antibiotic.endswith(" " + route):
            return route
    return ""  # Return empty string if no route is found.

# Apply the function to create a new 'Route' column.
df['Route'] = df['Antibiotic'].apply(extract_route)

# Remove any route names from the 'Antibiotic' column only if they appear at the end
# and are preceded by either ", " or " ".
for route in route_list:
    df['Antibiotic'] = df['Antibiotic'].str.replace(fr'(, | ){route}$', '', regex=True)


In [6]:
df[df['Antibiotic'].str.contains(r'[^A-Za-z ]', na=False)]

,Antibiotic,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,Route
2,Amoxicillin/ clavulanic-acid,Watch,Watch,
6,Benzathine-benzylpenicillin,Access,Access,
20,Ceftaroline-fosamil,Reserve,Reserve,
22,Ceftazidime/ avibactam,Reserve,Reserve,
23,Ceftobiprole-medocaril,Reserve,Reserve,
24,Ceftolozane/ tazobactam,Reserve,Reserve,
34,Dalfopristin/ quinupristin,Reserve,Reserve,
49,Imipenem/cilastatin,Reserve,Reserve,
50,Imipenem/cilastatin/relebactam,Reserve,Reserve,
55,Meropenem/ vaborbactam,Reserve,Reserve,


Some items contain a dash between words where as in dm+d there is a space. Replace the dash with a space to follow the common format.

In [7]:
df['Antibiotic'] = df['Antibiotic'].str.replace("-", " ", regex=False)

Some items contain a forward slash between words to indicate a combination where as in dm+d there is a + symbol. Replace the dash with a + ensuring appropriately spaced to follow the common dm+d format.

In [8]:
# Replace / with a + ensuring it is appropriately spaced and 2nd drug name is capitalised
df['Antibiotic'] = (
    df['Antibiotic']
    .str.replace(r'\s*/\s*', ' + ', regex=True)
    .str.replace(r'(\+\s+)(\w)', lambda m: m.group(1) + m.group(2).upper(), regex=True)
)
# Create a new column 'Combo' that is True if 'Antibiotic' contains a plus between two words.
df['Combo'] = df['Antibiotic'].str.contains(r'\b\w+\s*\+\s*\w+\b', regex=True)

In [9]:
df[df['Antibiotic'].str.contains(r'[^A-Za-z ]', na=False)]

,Antibiotic,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,Route,Combo
2,Amoxicillin + Clavulanic acid,Watch,Watch,,True
22,Ceftazidime + Avibactam,Reserve,Reserve,,True
24,Ceftolozane + Tazobactam,Reserve,Reserve,,True
34,Dalfopristin + Quinupristin,Reserve,Reserve,,True
49,Imipenem + Cilastatin,Reserve,Reserve,,True
50,Imipenem + Cilastatin + Relebactam,Reserve,Reserve,,True
55,Meropenem + Vaborbactam,Reserve,Reserve,,True
69,Piperacillin + Tazobactam,Watch,Watch,,True
77,Sulfamethoxazole + Trimethoprim,Access,Access,,True


Some antibiotic combinations are referred to as different names or in a different order in dm+d. Convert to use these. (Found these issues via dm+d manual search)

In [10]:
df['Antibiotic'] = df['Antibiotic'].str.replace("Amoxicillin + Clavulanic acid", "Co-amoxiclav", regex=False)
df['Antibiotic'] = df['Antibiotic'].str.replace("Sulfamethoxazole + Trimethoprim", "Co-trimoxazole", regex=False)
df['Antibiotic'] = df['Antibiotic'].str.replace("Dalfopristin + Quinupristin", "Quinupristin + Dalfopristin", regex=False)
df['Antibiotic'] = df['Antibiotic'].str.replace("Imipenem + Cilastatin + Relebactam", "Cilastatin + Imipenem + Relebactam", regex=False)
df['Antibiotic'] = df['Antibiotic'].str.replace("Ceftobiprole medocaril", "Ceftobiprole", regex=False)

Reorder the columns

In [11]:
df = df[['Antibiotic', 'Route', 'Combo'] + [col for col in df.columns if col not in ['Antibiotic', 'Route', 'Combo']]]

In [12]:
with pd.option_context('display.max_rows', None):
    display(df)

,Antibiotic,Route,Combo,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category
0,Amikacin,,False,Watch,Watch
1,Amoxicillin,,False,Access,Access
2,Co-amoxiclav,,True,Watch,Watch
3,Ampicillin,,False,Access,Access
4,Azithromycin,,False,Watch,Watch
5,Aztreonam,,False,Reserve,Reserve
6,Benzathine benzylpenicillin,,False,Access,Access
7,Benzylpenicillin,,False,Access,Access
8,Cefaclor,,False,Watch,Watch
9,Cefadroxil,,False,Watch,Access


### Matching to dm+d
Firstly try to match the antibiotic to a VTM

In [13]:
# Get a list of unique antibiotic values from your DataFrame.
list_of_abx = df['Antibiotic'].str.lower().unique().tolist()

# Convert the list into a comma-separated string of quoted values.
formatted_list = ", ".join(f"'{abx}'" for abx in list_of_abx)

# Construct the query using the formatted list.
sql = f"""
SELECT vtm.id, vtm.nm
FROM `ebmdatalab.dmd.vtm` vtm
WHERE lower(vtm.nm) IN ({formatted_list})
"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'vtm_match.csv')

# Use the cached_read function from the bq library to run the query.
vtms_df = bq.cached_read(sql, csv_path=csv_path)

# Rename vtms_df column 'id' to 'vtm_id'
vtms_df = vtms_df.rename(columns={'id': 'vtm_id'})

df = (
    df.assign(antibiotic_lower=df['Antibiotic'].str.lower())
      .merge(
          vtms_df.assign(nm_lower=vtms_df['nm'].str.lower()),
          left_on='antibiotic_lower',
          right_on='nm_lower',
          how='left'
      )
      .assign(vtm_id=lambda x: x['vtm_id'].fillna(0).astype('int64'))
      .drop(columns=['antibiotic_lower', 'nm_lower'])
)
df = df.rename(columns={'nm': 'vtm_nm'})

with pd.option_context('display.max_rows', None):
    display(df)

,Antibiotic,Route,Combo,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,vtm_id,vtm_nm
0,Amikacin,,False,Watch,Watch,774534000,Amikacin
1,Amoxicillin,,False,Access,Access,774586009,Amoxicillin
2,Co-amoxiclav,,True,Watch,Watch,774587000,Co-amoxiclav
3,Ampicillin,,False,Access,Access,774590006,Ampicillin
4,Azithromycin,,False,Watch,Watch,774722005,Azithromycin
5,Aztreonam,,False,Reserve,Reserve,774725007,Aztreonam
6,Benzathine benzylpenicillin,,False,Access,Access,1234764000,Benzathine benzylpenicillin
7,Benzylpenicillin,,False,Access,Access,774826003,Benzylpenicillin
8,Cefaclor,,False,Watch,Watch,775086009,Cefaclor
9,Cefadroxil,,False,Watch,Access,775087000,Cefadroxil


Show any rows where no VTM match has happened.

In [14]:
df[df['vtm_id'] == 0]

,Antibiotic,Route,Combo,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,vtm_id,vtm_nm
11,Cefalotin,,False,Watch,Access,0,NaN
68,Piperacillin,,False,Watch,Watch,0,NaN


After manual checking neither of these have an matching products in the UK currently so we can remove them.

In [15]:
df = df[df['vtm_id'] != 0]

Re-order the columns

In [16]:
df = df[['Antibiotic', 'Route', 'Combo', 'vtm_id'] + [col for col in df.columns if col not in ['Antibiotic', 'Route', 'Combo', 'vtm_id']]]

In [17]:
with pd.option_context('display.max_rows', None):
    display(df)

,Antibiotic,Route,Combo,vtm_id,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,vtm_nm
0,Amikacin,,False,774534000,Watch,Watch,Amikacin
1,Amoxicillin,,False,774586009,Access,Access,Amoxicillin
2,Co-amoxiclav,,True,774587000,Watch,Watch,Co-amoxiclav
3,Ampicillin,,False,774590006,Access,Access,Ampicillin
4,Azithromycin,,False,774722005,Watch,Watch,Azithromycin
5,Aztreonam,,False,774725007,Reserve,Reserve,Aztreonam
6,Benzathine benzylpenicillin,,False,1234764000,Access,Access,Benzathine benzylpenicillin
7,Benzylpenicillin,,False,774826003,Access,Access,Benzylpenicillin
8,Cefaclor,,False,775086009,Watch,Watch,Cefaclor
9,Cefadroxil,,False,775087000,Watch,Access,Cefadroxil


Seperate out items with and without specified route

In [18]:
# Ensure 'Route' is a string by replacing NaN with an empty string
df['Route'] = df['Route'].fillna('')

# Create df_no_route: rows where 'Route' is blank (empty or only whitespace)
df_no_route = df[df['Route'].str.strip() == '']

# Create df_route: rows where 'Route' has a value (non-blank)
df_route = df[df['Route'].str.strip() != '']

Get a list of VMPs from dm+d with VTMs in df_no_route - then match results onto df_no_route based on VTM code

In [19]:
# Get a list of unique antibiotic values from your DataFrame.
list_of_vtm = df_no_route['vtm_id'].unique().tolist()

# Convert the list into a comma-separated string of quoted values.
formatted_list = ", ".join(f"{vtm}" for vtm in list_of_vtm)

# Construct the query using the formatted list.
sql = f"""
SELECT
  vmp.vtm,
  vmp.id,
  vmp.nm,
  ontformroute.descr  
FROM `ebmdatalab.dmd.vmp` vmp
LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
LEFT JOIN dmd.ontformroute ontformroute ON ont.form = ontformroute.cd
WHERE vmp.vtm IN({formatted_list})
"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'vmp_match.csv')

# Use the cached_read function from the bq library to run the query.
vmps_df = bq.cached_read(sql, csv_path=csv_path)

# Rename vtms_df column 'id' to 'vtm_id'
vmps_df = vmps_df.rename(columns={'id': 'vmp_id'})

df_no_route = df_no_route.merge(vmps_df, left_on='vtm_id', right_on='vtm', how='right').drop(columns='vtm')
df_no_route = df_no_route.rename(columns={'nm': 'vmp_nm'})

Get a list of VMPs from dm+d with VTMs in df_route - then match results onto df_route based on VTM code and route

In [20]:
# Get a list of unique antibiotic values from your DataFrame.
list_of_vtm = df_route['vtm_id'].unique().tolist()

# Convert the list into a comma-separated string of quoted values.
formatted_list = ", ".join(f"{vtm}" for vtm in list_of_vtm)

# Construct the query using the formatted list.
sql = f"""
SELECT
  vmp.vtm,
  vmp.id,
  vmp.nm,
  ontformroute.descr  
FROM `ebmdatalab.dmd.vmp` vmp
LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
LEFT JOIN dmd.ontformroute ontformroute ON ont.form = ontformroute.cd
WHERE vmp.vtm IN({formatted_list})
"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'vmp_match2.csv')

# Use the cached_read function from the bq library to run the query.
vmps_df = bq.cached_read(sql, csv_path=csv_path)

# Rename vtms_df column 'id' to 'vtm_id'
vmps_df = vmps_df.rename(columns={'id': 'vmp_id'})

# Add a column route for matching on
vmps_df['route'] = vmps_df['descr'].str.split(r'\.').str[1]

# Match VMPs
df_route = df_route.merge(
    vmps_df,
    left_on=['vtm_id', 'Route'],
    right_on=['vtm', 'route'],
    how='left'
).assign(vmp_id=lambda x: x['vmp_id'].fillna(0).astype('int64')).drop(columns=['vtm', 'route'])

df_route = df_route.rename(columns={'nm': 'vmp_nm'})

Combine df_route and df_no_route back together.

In [21]:
df_combined = pd.concat([df_route, df_no_route], ignore_index=True)

Check for rows without a VMP match

In [22]:
df_combined[df_combined['vmp_id'] == 0]

,Antibiotic,Route,Combo,vtm_id,England-adapted 2019 AWaRe category,UK-adapted 2024 AWaRe category,vtm_nm,vmp_id,vmp_nm,descr
27,Streptomycin,intravenous,False,777614004,Other,Other,Streptomycin,0,NaN,NaN


On manual check Streptomycin 1g powder for solution for injection vials is the only VMP and route is given only as intramuscular-deep so remove from list.

In [23]:
df_combined = df_combined[df_combined['vmp_id'] != 0]

Tidy up results

In [24]:
# Define the desired columns to appear first
cols_first = ['Antibiotic', 'Route', 'Combo', 'vtm_nm', 'vtm_id', 'vmp_nm', 'vmp_id', 'descr']

# Get the list of remaining columns that are not in cols_first
other_cols = [col for col in df_combined.columns if col not in cols_first]

# Reorder the DataFrame columns
df_combined = df_combined[cols_first + other_cols]

# Sort the DataFrame
df_combined = df_combined.sort_values(['Antibiotic', 'vtm_nm', 'vmp_nm'])

# Save to CSV
df_combined.to_csv('AWaRe_VMPs.csv')